In [1]:
# Qiao's first Spark Notebook
from pyspark import SparkContext
quote = "To be, or not to be, that is the question: Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune, Or to take Arms against a Sea of troubles, And by opposing end them: to die, to sleep No more; and by a sleep, to say we end the heart-ache, and the thousand natural shocks that Flesh is heir to? 'Tis a consummation devoutly to be wished. To die, to sleep, To sleep, perchance to Dream; aye, there's the rub, for in that sleep of death, what dreams may come, when we have shuffled off this mortal coil, must give us pause."

In [2]:
sparkdata = sc.parallelize(quote.split(' '))

In [3]:
print ("sparkdata = ", sparkdata)
print ("sparkdata.collect = ", sparkdata.collect)
print ("sparkdata.collect() = ", sparkdata.collect()[1:10])

sparkdata = ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:330
sparkdata.collect = <bound method RDD.collect of ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:330>
sparkdata.collect() = ['be,', 'or', 'not', 'to', 'be,', 'that', 'is', 'the', 'question:']

In [4]:
print(sparkdata)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:330

In [5]:
print ("sparkdata.collect() = ", sparkdata.collect()[1:20])

sparkdata.collect() = ['be,', 'or', 'not', 'to', 'be,', 'that', 'is', 'the', 'question:', 'Whether', "'tis", 'nobler', 'in', 'the', 'mind', 'to', 'suffer', 'The', 'slings']

In [6]:
# Asimple transformation - map
def mapword(word):
  return (word, 1)

print (sparkdata.map(mapword)) # Nothing has happened here
print (sparkdata.map(mapword).collect()[0:10]) # Uppercase and lowercase letters are seperated;

PythonRDD[3] at RDD at PythonRDD.scala:57
[('To', 1), ('be,', 1), ('or', 1), ('not', 1), ('to', 1), ('be,', 1), ('that', 1), ('is', 1), ('the', 1), ('question:', 1)]

In [7]:
# Another Transformation
def charsmorethan2(tuple1):
  if len(tuple1[0]) > 2:
    return tuple1
rdd3 = sparkdata.map(mapword).filter(lambda x:charsmorethan2(x))


In [8]:
rdd3.collect()[0:10]


Out[ 25 ]: [('be,', 1),
 ('not', 1),
 ('be,', 1),
 ('that', 1),
 ('the', 1),
 ('question:', 1),
 ('Whether', 1),
 ("'tis", 1),
 ('nobler', 1),
 ('the', 1)]

In [9]:
# 2ND Example: Tables
cms = sc.parallelize([[1,"Dr. A",12.50,"Yale"],[2,"Dr.B",5.10,"Duke"],[3,"Dr.C",200.34,"Mt. Sinai"],[4,"Dr.D",5.67,"Duke"],[1,"Dr. E",52.50,"Yale"]])

In [10]:
def findPayment(data):
  return data[2]
print ("Payments = ", cms.map(findPayment).collect())
print ("Mean = ", cms.map(findPayment).mean())

Payments = [12.5, 5.1, 200.34, 5.67, 52.5]
Mean = 55.222

In [11]:
# Creating a dataframe
from pyspark.sql.functions import avg, max, min
cms_df = sqlContext.createDataFrame(cms, ["ID", "Name", "Payment", "Hosp"])
print (cms_df.show())
print (cms_df.groupby('Hosp').agg(avg('Payment'), max('Payment'),min('Payment')))

+---+-----+-------+---------+
 ID| Name|Payment| Hosp|
+---+-----+-------+---------+
 1|Dr. A| 12.5| Yale|
 2| Dr.B| 5.1| Duke|
 3| Dr.C| 200.34|Mt. Sinai|
 4| Dr.D| 5.67| Duke|
 1|Dr. E| 52.5| Yale|
+---+-----+-------+---------+

None
DataFrame[Hosp: string, avg(Payment): double, max(Payment): double, min(Payment): double]

In [12]:
print (cms_df.groupby('Hosp').agg(avg('Payment'), max('Payment'),min('Payment')).collect())

[Row(Hosp='Duke', avg(Payment)=5.385, max(Payment)=5.67, min(Payment)=5.1), Row(Hosp='Yale', avg(Payment)=32.5, max(Payment)=52.5, min(Payment)=12.5), Row(Hosp='Mt. Sinai', avg(Payment)=200.34, max(Payment)=200.34, min(Payment)=200.34)]

In [13]:
print("Converting to a Pandas DataFrame")
print("-----------------------------------")
pd_df = cms_df.groupby('Hosp').agg(avg('Payment'), max('Payment'),min('Payment')).toPandas()
print (type(pd_df))
print(pd_df)

Converting to a Pandas DataFrame
-----------------------------------
<class 'pandas.core.frame.DataFrame'>
 Hosp avg(Payment) max(Payment) min(Payment)
0 Duke 5.385 5.67 5.10
1 Yale 32.500 52.50 12.50
2 Mt. Sinai 200.340 200.34 200.34

In [14]:
wordsList = ['to', 'be', 'or', 'not', 'to', 'be']
wordsRDD = sc.parallelize(wordsList, 3)
print(type(wordsRDD))
print(wordsRDD)

<class 'pyspark.rdd.RDD'>
ParallelCollectionRDD[61] at readRDDFromFile at PythonRDD.scala:330

In [15]:
print(wordsRDD.glom().take(2))
print(wordsRDD.glom().collect())

[['to', 'be'], ['or', 'not']]
[['to', 'be'], ['or', 'not'], ['to', 'be']]

In [16]:
def makeUpperCase(word):
  return word.upper()
print(makeUpperCase('cat'))

CAT

In [17]:
upperRDD = wordsRDD.map(makeUpperCase)
print (upperRDD.collect())
upperLambdaRDD = wordsRDD.map(lambda word: word.upper())
print(upperLambdaRDD.collect())

['TO', 'BE', 'OR', 'NOT', 'TO', 'BE']
['TO', 'BE', 'OR', 'NOT', 'TO', 'BE']

In [18]:
wordPairs = wordsRDD.map(lambda word: (word, 1))
print (wordPairs.collect())

[('to', 1), ('be', 1), ('or', 1), ('not', 1), ('to', 1), ('be', 1)]

In [19]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
  print ('{0}:{1}'.format(key, list(value)))

not:[1]
or:[1]
to:[1, 1]
be:[1, 1]

In [20]:
wordCounts = wordPairs.reduceByKey(lambda a,b: a+b)
print(wordCounts.collect())

[('not', 1), ('or', 1), ('to', 2), ('be', 2)]

In [21]:
wordCountsCollected = (wordsRDD.map(lambda word: (word,1)).reduceByKey(lambda a,b:a+b).collect())
print(wordCountsCollected)

[('not', 1), ('or', 1), ('to', 2), ('be', 2)]